In [2]:
import os
import collections
import pandas as pd
import sqlalchemy as sql
import datetime as dt

In [3]:
LOANS_DIR = '/p2p_lending/prosper/loans'
LISTINGS_DIR = '/p2p_lending/prosper/listings'

## Preparation

In [4]:
def payment(amount,rate,periods):
    '''Calculate loan payment, given principal amount, monthly interest rate, and term (periods).'''
    increase    = (1+rate)**periods
    if abs(increase - 1) < 10**(-10):
        return 0
    pmt         = rate*amount*increase/float(increase - 1)
    return pmt

### Get Samples for Work on Column Setup

In [5]:
seed = 324

In [ ]:
loan_sample = pd.read_csv(os.path.join(LOANS_DIR,
                                 'Loans_20140101to20150101_20161007T060016.csv'),
                                 header=0,
                                 low_memory=True).sample(10000,random_state=seed)

In [6]:
listing_sample = pd.read_csv(os.path.join(LISTINGS_DIR,
                                 'Listings_20150101to20160101_20161007T060524.csv'),
                                 header=0,
                                 low_memory=True).sample(10000,random_state=seed)

#note samples retain indices, so they won't count normally
# for example loans.ix[0] returns an error if item 0 is not in the sample 

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (535) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Add a payment column and simple return column

In [5]:
loan_sample['payment'] = loan_sample.apply(lambda row: payment(row.amount_borrowed,row.borrower_rate/12.,row.term),axis=1)
loan_sample['simple_return'] = (loan_sample.interest_paid + loan_sample.principal_paid)/loan_sample.amount_borrowed

#### Get rid of columns without any data or without any variation

In [6]:
loan_sample.count().sort_values().head()

loan_default_reason_description      792
loan_default_reason                  792
next_payment_due_date               9928
loan_number                        10000
next_payment_due_amount            10000
dtype: int64

In [7]:
listing_sample.count().sort_values().head()

group_name                    0
verification_stage            0
prior_prosper_loans61dpd      2
prior_prosper_loans31dpd      3
whole_loan_end_date         135
dtype: int64

Documentation on the fields indicates that there are several fields which are deprecated. 
- group_name
- group_indicator
- borrower_listing_description
- channel_code
- listing_purpose

It makes sense to get rid of these fields since they won't be useful for current/future loans. Actually, some of them have already been removed in the data. The verification_stage field is not deprecated, but it appears to always be 0, which isn't helpful to us.

In [8]:
del listing_sample['group_indicator'], listing_sample['group_name']
del listing_sample['channel_code']

#### Get rid of fields without variation

In [9]:
listing_sample.apply(lambda col: pd.value_counts(col).count(), axis=0).sort_values().head(10)

verification_stage            0
prior_prosper_loans31dpd      1
prior_prosper_loans61dpd      1
amount_participation          1
borrower_metropolitan_area    1
gbl007                        1
rev113                        2
listing_term                  2
iln113                        2
rev110                        2
dtype: int64

In [10]:
print 'loans31dpd: ', listing_sample.prior_prosper_loans31dpd.head(1).values
print 'amount_participation: ', listing_sample.amount_participation.head(1).values
print 'borrower_metro: ', listing_sample.borrower_metropolitan_area.head(1).values
print 'verification_stage: ', listing_sample.verification_stage.head(1).values
print 'loans61dpd: ', listing_sample.prior_prosper_loans61dpd.head(1).values
print 'gbl007: ', listing_sample.gbl007.head(1).values

loans31dpd:  [ nan]
amount_participation:  [0]
borrower_metro:  ['(Not Implemented)']
verification_stage:  [ nan]
loans61dpd:  [ nan]
gbl007:  [ 0.]


In [11]:
del listing_sample['prior_prosper_loans31dpd'], listing_sample['prior_prosper_loans61dpd']
del listing_sample['amount_participation'], listing_sample['borrower_metropolitan_area']
del listing_sample['gbl007'], listing_sample['verification_stage']

#### Fix Dates from string type to datetime type

The oldest_trade_open_date column has multiple formats within some files (i.e. 08231992, '8/25/200', '9/19/199, 1, 2, NaN); this breaks the pd.to_datetime function.
Therefore, I have decided to get rid of this column.

In [14]:
del listing_sample['oldest_trade_open_date']

In [ ]:
for col in loan_sample.columns:
    if 'date' in col:
        loan_sample[col] = pd.to_datetime(loan_sample[col])

In [15]:
for col in listing_sample.columns:
    if 'date' in col:
        listing_sample[col] = pd.to_datetime(listing_sample[col])

In [16]:
listing_sample['first_recorded_credit_line'] = pd.to_datetime(listing_sample['first_recorded_credit_line'])

#### Set up column types

In [55]:
ln_col_types = loan_sample.apply(lambda col: collections.Counter(
                            [type(i) for i in col if type(i)!=object]).most_common(1)[0][0])
 

In [17]:
listing_col_types = listing_sample.apply(lambda col: collections.Counter(
                            [type(i) for i in col if type(i)!=object]).most_common(1)[0][0])         

In [18]:
chking = listing_sample.apply(lambda col: set(col.apply(type)))
chking[chking.apply(len) > 1]

listing_end_date                 {<class 'pandas.tslib.Timestamp'>, <class 'pan...
loan_origination_date            {<class 'pandas.tslib.Timestamp'>, <class 'pan...
scorex                                              {<type 'str'>, <type 'float'>}
scorex_change                                       {<type 'str'>, <type 'float'>}
employment_status_description                       {<type 'str'>, <type 'float'>}
occupation                                          {<type 'str'>, <type 'float'>}
whole_loan_start_date            {<class 'pandas.tslib.Timestamp'>, <class 'pan...
whole_loan_end_date              {<class 'pandas.tslib.Timestamp'>, <class 'pan...
dtype: object

In [19]:
listing_col_types['scorex_change'] = str    #most of these are np.nan, but correct type is str
listing_col_types['scorex'] = str
listing_col_types['employment_status_description'] = str
listing_col_types['occupation'] =str

In [20]:
for col in listing_col_types.index:
    if 'date' in col:
        listing_col_types[col] = pd.tslib.Timestamp

In [16]:
ln_col_types[ln_col_types==str]

prosper_rating             <type 'str'>
loan_status_description    <type 'str'>
dtype: object

In [21]:
listing_col_types[listing_col_types==str]

member_key                       <type 'str'>
listing_status_reason            <type 'str'>
prosper_rating                   <type 'str'>
scorex                           <type 'str'>
scorex_change                    <type 'str'>
fico_score                       <type 'str'>
income_range_description         <type 'str'>
employment_status_description    <type 'str'>
occupation                       <type 'str'>
borrower_state                   <type 'str'>
borrower_city                    <type 'str'>
investment_type_description      <type 'str'>
dtype: object

In [18]:
print pd.unique(ln_col_types)

[<type 'int'> <type 'float'> <type 'str'> <class 'pandas.tslib.Timestamp'>]


In [18]:
print pd.unique(listing_col_types)

[<type 'str'> <type 'int'> <class 'pandas.tslib.Timestamp'> <type 'float'>
 <type 'bool'>]


To convert a DataFrame to an sql table, we will need to use sqlalchemy types. A conversion table will help with this.

In [22]:
type_cvert = {int: sql.types.Integer,
              float: sql.types.Float,
              pd.tslib.Timestamp: sql.types.Date, 
              bool: sql.types.Boolean,
              pd.tslib.NaTType: sql.types.NullType}

#### Loan Columns: Convert to sqlalchemy types

In [21]:
ln_col_types

loan_number                                            <type 'int'>
amount_borrowed                                      <type 'float'>
borrower_rate                                        <type 'float'>
prosper_rating                                         <type 'str'>
term                                                   <type 'int'>
age_in_months                                          <type 'int'>
origination_date                   <class 'pandas.tslib.Timestamp'>
days_past_due                                          <type 'int'>
principal_balance                                    <type 'float'>
service_fees_paid                                    <type 'float'>
principal_paid                                       <type 'float'>
interest_paid                                        <type 'float'>
prosper_fees_paid                                    <type 'float'>
late_fees_paid                                       <type 'float'>
debt_sale_proceeds_received                     

#### Fix loan_default_reason_description column to be varchar

In [57]:
import numpy as np
ln_col_types.loan_default_reason_description = str
#listing_sample.ix[listing_sample.scorex.apply(type)==float, listing_col_types.index[listing_col_types==str]]
str_columns = ln_col_types.index[ln_col_types==str]
fixna_dict = dict.fromkeys(str_columns,'')
fixna = loan_sample.fillna(fixna_dict, inplace=True)

In [58]:
for ind in ln_col_types.index:
    if ln_col_types[ind] != str:
        ln_col_types[ind] = type_cvert[ln_col_types[ind]]
    else:
        ln_col_types[ind] = sql.types.String(loan_sample[ind].apply(len).max())

In [59]:
ln_col_types

loan_number                        <class 'sqlalchemy.sql.sqltypes.Integer'>
amount_borrowed                      <class 'sqlalchemy.sql.sqltypes.Float'>
borrower_rate                        <class 'sqlalchemy.sql.sqltypes.Float'>
prosper_rating                                                    VARCHAR(2)
term                               <class 'sqlalchemy.sql.sqltypes.Integer'>
age_in_months                      <class 'sqlalchemy.sql.sqltypes.Integer'>
origination_date                      <class 'sqlalchemy.sql.sqltypes.Date'>
days_past_due                      <class 'sqlalchemy.sql.sqltypes.Integer'>
principal_balance                    <class 'sqlalchemy.sql.sqltypes.Float'>
service_fees_paid                    <class 'sqlalchemy.sql.sqltypes.Float'>
principal_paid                       <class 'sqlalchemy.sql.sqltypes.Float'>
interest_paid                        <class 'sqlalchemy.sql.sqltypes.Float'>
prosper_fees_paid                    <class 'sqlalchemy.sql.sqltypes.Float'>

#### Listing Columns: Convert to sqlalchemy types

The str type columns had some NaN values, which were of type float. I replace these with the empty string. 

In [23]:
import numpy as np
#listing_sample.ix[listing_sample.scorex.apply(type)==float, listing_col_types.index[listing_col_types==str]]
str_columns = listing_col_types.index[listing_col_types==str]
fixna_dict = dict.fromkeys(str_columns,'')
fixna = listing_sample.fillna(fixna_dict, inplace=True)

In [24]:
for ind in listing_col_types.index:
    if listing_col_types[ind] != str:
        listing_col_types[ind] = type_cvert[listing_col_types[ind]]
    elif ind in ['borrower_city','borrower_metropolitan_area']:
        listing_col_types[ind] = sql.types.String(int(listing_sample[ind].apply(len).max()*1.6))
    else:
        listing_col_types[ind] = sql.types.String(listing_sample[ind].apply(len).max())

In [25]:
listing_col_types

member_key                                                     VARCHAR(23)
listing_number                   <class 'sqlalchemy.sql.sqltypes.Integer'>
credit_pull_date                    <class 'sqlalchemy.sql.sqltypes.Date'>
listing_start_date                  <class 'sqlalchemy.sql.sqltypes.Date'>
listing_end_date                    <class 'sqlalchemy.sql.sqltypes.Date'>
listing_creation_date               <class 'sqlalchemy.sql.sqltypes.Date'>
loan_origination_date               <class 'sqlalchemy.sql.sqltypes.Date'>
listing_status                   <class 'sqlalchemy.sql.sqltypes.Integer'>
listing_status_reason                                           VARCHAR(9)
listing_amount                     <class 'sqlalchemy.sql.sqltypes.Float'>
amount_funded                      <class 'sqlalchemy.sql.sqltypes.Float'>
amount_remaining                   <class 'sqlalchemy.sql.sqltypes.Float'>
percent_funded                     <class 'sqlalchemy.sql.sqltypes.Float'>
partial_funding_indicator

As shown below, there are 102 duplicate listing_number entries. After investigation, these represent the same listings (recorded multiple times). 

In [26]:
listing_files = os.listdir(LISTINGS_DIR)

for i,f in enumerate(listing_files):
    if i ==0:
        df = pd.read_csv(os.path.join(LISTINGS_DIR,f), header=0, low_memory=True, usecols=['listing_number','borrower_rate',
                                                                                          'prosper_rating','estimated_return',
                                                                                          'loan_origination_date','amount_funded'])
    else:
        df.append(pd.read_csv(os.path.join(LISTINGS_DIR,f), header=0, low_memory=True, usecols=['listing_number','borrower_rate',
                                                                                          'prosper_rating','estimated_return',
                                                                                          'loan_origination_date','amount_funded']))

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
df[df.duplicated(subset=['listing_number'])].listing_number.count()

102

### Database Connection

In [28]:
creds = '<username>:<password>'

In [29]:
eng = sql.create_engine('postgresql://%s@localhost:5432/p2p_lending' %creds)

In [30]:
con = eng.connect()

### Loans Data Ingest

#### Create table

In [31]:
from sqlalchemy.ext.declarative import declarative_base

In [61]:
Base = declarative_base()

In [62]:
class Loans(Base):
    __tablename__ = 'loans'
    for ind in ln_col_types.index:
        if ind == 'loan_number':
            sql_type = ln_col_types[ind]
            exec '%s = sql.Column(sql_type, primary_key=True)' %(ind)
        else:
            sql_type = ln_col_types[ind]
            exec '%s = sql.Column(sql_type)' %(ind)

In [63]:
Base.metadata.create_all(con)

#### Ingest Data

In [64]:
sqlalchemy = sql

loan_files = os.listdir(LOANS_DIR)

for f in loan_files:
    #read data
    loans = pd.read_csv(os.path.join(LOANS_DIR,f), header=0, low_memory=True)    
    #add new columns
    loans['payment'] = loans.apply(lambda row: payment(row.amount_borrowed,row.borrower_rate/12.,row.term),axis=1)
    loans['simple_return'] = (loans.interest_paid + loans.principal_paid)/loans.amount_borrowed
    #fix date columns
    for col in loans.columns:
        if 'date' in col:
            loans[col] = pd.to_datetime(loans[col])
    #write data to sql table (using sql converted types)
    loans.to_sql(name='loans', con=con, if_exists='append', index=False, dtype=dict(ln_col_types))

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Listings Data Ingest

In [32]:
Base = declarative_base()

In [33]:
class Listings(Base):
    __tablename__ = 'listings'
    for ind in listing_col_types.index:
        if ind == 'listing_number':
            sql_type = listing_col_types[ind]
            exec '%s = sql.Column(sql_type, primary_key=True)' %(ind)
        else:
            sql_type = listing_col_types[ind]
            exec '%s = sql.Column(sql_type)' %(ind)

In [34]:
Base.metadata.create_all(con)

In [37]:
listing_files = os.listdir(LISTINGS_DIR)
listing_numbers = set()
for f in listing_files:
    print 'starting file %s' %f
    #read data
    listings = pd.read_csv(os.path.join(LISTINGS_DIR,f), header=0, low_memory=True)
    #delete deprecated and unimportant columns
    del listings['group_indicator'], listings['group_name']
    del listings['channel_code']
    del listings['prior_prosper_loans31dpd'], listings['prior_prosper_loans61dpd']
    del listings['amount_participation'], listings['borrower_metropolitan_area']
    del listings['gbl007'], listings['verification_stage']    
    #fix date columns
    del listings['oldest_trade_open_date']
    for col in listings.columns:
        if 'date' in col:
            listings[col] = pd.to_datetime(listings[col])
    listings['first_recorded_credit_line'] = pd.to_datetime(listings['first_recorded_credit_line'])
    #check for duplicate listing_numbers (there are 181)
    
    listings = listings.drop_duplicates(subset=['listing_number'])    #get rid of duplicates within file
    current_listing_n = set(listings.listing_number)
    dups = listing_numbers & current_listing_n
    listing_numbers = listing_numbers | current_listing_n
    #get rid of duplicates accross files
    if dups:
        listings = listings[~listings.listing_number.isin(dups)]
    #write data to sql table
    listings.to_sql(name='listings', con=con, if_exists='append', index=False,
                    dtype=dict(listing_col_types), chunksize=7000)

starting file Listings_20160101toCurrent_20161007T061001.csv
starting file Listings_20050101to20130101_20161007T060003.csv


/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (16,25,26,27,531,535,538) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


starting file Listings_20130101to20140101_20161007T060218.csv


/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (26,33,57,531,535,538,539) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


starting file Listings_20140101to20150101_20161007T060259.csv


/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (33,535) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


starting file Listings_20150101to20160101_20161007T060524.csv


In [38]:
print 'done!'

done!


In [39]:
con.close()